## 测试GPU可用性

In [ ]:
import tensorrt as trt
import tensorflow as tf
print(tf.config.list_physical_devices('GPU'))

In [ ]:
import torch
# CUDA
print(torch.cuda.is_available())
# cuDNN 深度学习加速
print(torch.backends.cudnn.enabled)
a = torch.rand(5).to('cuda')
a

## 启用代理（AutoDL）

In [ ]:
import subprocess
import os

result = subprocess.run('bash -c "source ~/clash_dir/set && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
output
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

# 情感分析


## 默认模型

In [ ]:
import tensorrt as trt
from transformers import pipeline
classifier = pipeline(task="sentiment-analysis")

In [ ]:
title = "Intelligent Systems wins bullish rating from BTIG on Apple Card role"

In [ ]:
content = """
BTIG analyst Mark Palmer initiates coverage of small cap Intelligent Systems  INS  0 3   with Buy recommendation for its role in helping Goldman Sachs serve as processor for the launch of the Apple Card 
 We believe INS is an overlooked small cap stock with compelling growth prospects   Palmer writes in a note 
Sees INS s growth to be slower in 2020  though still very strong  as it executes on the Apple Card project   We view its long term growth outlook as attractive and supportive of a premium multiple   he said 
Also sees Apple Card s success making it more likely that Goldman will use INS s CoreCard software for processing projects with other firms in the future 
Quant rating is Bullish
"""

In [ ]:
preds = classifier(content)
preds = [{"score": round(pred["score"], 4), "label": pred["label"]} for pred in preds]
preds

## FinBERT-tone

In [ ]:
sequences = [
    "I've been waiting for a HuggingFace course my whole life.", 
    "So have I!"
]

In [ ]:
import tensorrt as trt
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import pipeline

checkpoint = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
nlp = pipeline("sentiment-analysis",
               model=checkpoint,
               tokenizer=tokenizer,
               max_length=512,
               truncation=True,
               device=0)
pl_result = nlp(sequences)
pl_result

In [ ]:
import torch
from transformers import AutoModel,BertTokenizer,BertForSequenceClassification

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

checkpoint = BertForSequenceClassification.from_pretrained('yiyanghkust/finbert-tone',num_labels=3).to(device)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')

tokens = tokenizer(sequences, 
                   padding=True,
                   truncation=True, 
                   max_length=512,
                   return_tensors="pt").to(device)
# print(tokens)
output = checkpoint(**tokens)
logits = output.logits

labels = checkpoint.config.id2label
prob = torch.nn.functional.softmax(logits, dim=-1)
preds = logits.argmax(dim=-1)
probs = prob[torch.arange(logits.shape[0]), preds]
results = [{'label': labels[pred.item()], 'score': prob.item()} for pred, prob in zip(preds, probs)]
results

## sample实验

In [ ]:
sentences = ["there is a shortage of capital, and we need extra financing",  
             "growth is strong and we have plenty of liquidity", 
             "there are doubts about our finances", 
             "profits are flat"]
results = nlp(sentences)
print(results) 

In [ ]:
content = """
BTIG analyst Mark Palmer initiates coverage of small cap Intelligent Systems  INS  0 3   with Buy recommendation for its role in helping Goldman Sachs serve as processor for the launch of the Apple Card 
 We believe INS is an overlooked small cap stock with compelling growth prospects   Palmer writes in a note 
Sees INS s growth to be slower in 2020  th|ough still very strong  as it executes on the Apple Card project   We view its long term growth outlook as attractive and supportive of a premium multiple   he said 
Also sees Apple Card s success making it more likely that Goldman will use INS s CoreCard software for processing projects with other firms in the future 
Quant rating is Bullish
"""
results = nlp(content)
results

In [ ]:
header = """
Apple Buys Another Artificial Intelligence Start Up
"""

content = """
It s been about a month since the last Apple  NASDAQ AAPL  acquisition that investors have caught wind of  Spectral Edge  to bolster the company s computational photography   CEO Tim Cook confirmed last year that the tech titan tends to buy smaller companies every two to three weeks  Apple s latest acquisition is all about low power edge based artificial intelligence  AI  
Here s what investors need to know 

Doing things locally
GeekWire reports that Apple has scooped up Xnor ai  a Seattle based start up that specializes in on device AI  Xnor ai was previously spun out of Microsoft co founder Paul Allen s AI incubator  the Allen Institute for AI  AI2   back in 2017  The company provides tools through a self service platform that allow other developers to incorporate AI into their apps 
Apple paid an estimated  200 million for Xnor ai  according to the report  Apple had previously acquired another Seattle based AI start up  Turi  back in 2016  also for around  200 million 

Xnor ai s main differentiator is that its technology keeps all of the AI processing on the local device as opposed to relying on data centers in the cloud  Apple prefers to store as much data locally as possible in order to improve privacy protections 
Apple s A series of chips include a neural engine that handles most of the machine learning and AI locally on iPhones  with the latest A13 Bionic chip boasting significant machine learning performance improvements thanks to its eight core neural engine  Sensitive biometric data used for Face ID or Touch ID is also stored locally in what Apple calls a Secure Enclave 
At the same time  low power computing is critical to improving battery life  The start up showed off an AI chip last year that used so little energy  it could run off solar power 
In recent years  Apple has been aggressively expanding in Seattle  The company said last summer that the Emerald City would  become a key engineering hub  that would employ 2 000 workers within the next few years  The Mac maker s office is right around the corner from Amazon s headquarters  and Apple is undoubtedly poaching talent from its rival 
Sorry  Wyze
Xnor ai s tech was incorporated into Wyze s smart home security cameras last summer  enabling people detection on the affordable    the entry level Wyze Cam V2 costs just  20    devices that was all processed locally  Just a few months after that feature was added  Wyze said in November that the people detection capability would be removed in January 2020 after Xnor ai terminated its contract 
Wyze is internally developing a comparable feature  although it may use the cloud instead of on device processing  The timing of all of that probably isn t a coincidence  It s entirely possible that Apple bought Xnor ai a couple months ago and investors are just now finding out
"""

In [ ]:
result = nlp(header)
result

In [ ]:
def sliding_window(tokenizer, text, max_len, stride):
    """将文本分割成指定大小的窗口，每个窗口有一定的重叠部分"""
    tokens = tokenizer.tokenize(text)
    token_windows = []
    for i in range(0, len(tokens), stride):
        window = tokens[i:i+max_len]
        if len(window) > max_len:
            window = window[:max_len]
        token_windows.append(window)
    return token_windows

# 将内容分割成不超过512个令牌的窗口，每个窗口有128个令牌的重叠部分
windows = sliding_window(tokenizer, content, 510, 384)

# 对每个窗口进行预测
results = []
for window in windows:
    window_text = tokenizer.decode(tokenizer.convert_tokens_to_ids(window))
    result = nlp(window_text)
    results.append(result)

In [ ]:
results

In [ ]:
for window in windows:
    print(len(window))

In [ ]:
result = nlp(content)
result